In [43]:
import csv
import copy
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [44]:
from feature import Feature
import sys
sys.dont_write_bytecode = True
sys.path.append('../')
from tool.maneger import DataManager

In [45]:
path = "../corpus/data_augment.csv"

In [46]:
data = []
with open(path) as f:
    reader = csv.reader(f)
    for row in reader:
        data.append(row)

In [47]:
def make_classes_dict(data):
    classes_dict = dict()
    for row in data:
        c = row[1]
        if c not in classes_dict:
            classes_dict[c] = len(classes_dict)
    return classes_dict

In [48]:
def class_filter(data, remain_classes:list):
    new_data = []
    for row in data:
        c = row[1]
        if c in remain_classes:
            new_data.append(row)
    return new_data

In [49]:
classes_dict_ = make_classes_dict(data)

In [50]:
remain_classes = "how what when where who why YN plain".split()
data_n = class_filter(data, remain_classes)
classes_dict = dict(zip(remain_classes, list(range(len(remain_classes)))))

In [51]:
def extract_X_y(data, classes_dict):
    X = []
    y = []
    for d in data:
        X.append(d[0])
        y.append(classes_dict[d[1]])
    return X, y

In [52]:
import random
def extract_X_y_limit(data, classes_dict:dict, limit=50):
    data_ = random.sample(data, len(data))
    X = []
    y = []
    each_len = dict(zip(classes_dict.keys(), [0]*len(classes_dict)))
    for d in data_:
        if each_len[d[1]] <= limit:
            X.append(d[0])
            y.append(classes_dict[d[1]])
            each_len[d[1]] += 1
    return X, y

In [53]:
# X, y = extract_X_y(data_n, classes_dict)

In [54]:
X, y = extract_X_y_limit(data_n, classes_dict)

In [55]:
len(X)

408

In [56]:
X_train_str, X_test_str, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=5)

In [57]:
F = Feature()
F.make_features(X_train_str)

X_train = []
X_test = []
for i, x_t_str in enumerate( X_train_str ):
    x = F.featurization(x_t_str)
    X_train.append(x)
for i, x_t_str in enumerate( X_test_str ):
    x = F.featurization(x_t_str)
    X_test.append(x)
X_train = np.array(X_train)
X_test = np.array(X_test)

300


In [58]:
F_path = "../X_y_data/"
F_name = "typeClassify_F2.dill"
import dill
# featureM = DataManager(F_path, format_="dill")
with open(F_path+F_name, "wb") as f:
    dill.dump(F, f)

In [59]:
lr = LogisticRegression(solver='sag', max_iter=1000)
lr.fit(X_train, y_train)

LogisticRegression(max_iter=1000, solver='sag')

In [60]:
y_pred = lr.predict(X_test)

In [61]:
print(classes_dict)
dis = 30
for y_p, x_s in zip(y_pred[:dis], X_test_str[:dis]):
    print("{0} : {1}".format(y_p, x_s))

{'how': 0, 'what': 1, 'when': 2, 'where': 3, 'who': 4, 'why': 5, 'YN': 6, 'plain': 7}
1 : みなさん乗った？
2 : 劇の稽古はいつ？
1 : ぼくは何をすればいい？
1 : 何だって？
4 : これはだれのノートかな？
5 : どうして君たちは仲良くできないんだい？
7 : 実は、わたしは走ることがあまり好きではないんですよ。
1 : 何で、ロボコーパスはいなくなったの？
5 : ドン、あなたの弟にとって、あなたはあまり気を遣う必要がない人なのよ、そうは思わない？
6 : 聞いてもいいですか？
3 : マリーナ駅はどこですか？
7 : 我々は年度末までに何を達成したいのか、考えてみましょう。
7 : 学生時代に、日本で外国人旅行客のツアーガイドのアルバイトをしていた時、最もよく質問されたことの1つが、「向こうにいるあの人たちはなぜマスクをしているのですか。
5 : とにかく、いつ帰ってくるの？
5 : なぜ、「commencement」と呼ばれるんだろう？
5 : 誰が、このお知らせを掲示板に載せたの？
1 : 誰がそのおいしそうなチョコレートケーキを作ったの？
2 : 提出期限はいつですか。
7 : 私にはいろいろなことがわかります。
5 : おばあさんに電話してあげたらどうなの？
3 : 巻き戻しボタンはどこかしら？
2 : そのコンサートはいつあったの？
0 : 「おお、ロミオ、ロミオ！　あなたはなぜロミオなの？」
5 : なぜ…なぜもっと早くわたしに言わなかったの？
3 : どこの出身なの？
0 : 体調はどうですか？
5 : ピンクベリー・キャンディーのほかのメンバーも誘ってもらえませんか？
6 : ＣＤ売り場はありますか？
1 : そのパーティーのことをどうやって知ったのですか。
1 : どういう意味かな？


In [62]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
print('confusion matrix = \n', confusion_matrix(y_true=y_test, y_pred=y_pred))
print('accuracy = ', accuracy_score(y_true=y_test, y_pred=y_pred))

confusion matrix = 
 [[ 6  4  0  0  0  1  3  0]
 [ 0  6  0  0  3  0  0  1]
 [ 1  1 11  0  0  1  0  1]
 [ 0  0  0 16  2  0  1  0]
 [ 1  1  2  0 14  1  1  1]
 [ 1  1  0  0  0 11  0  0]
 [ 0  3  0  1  2  2  7  0]
 [ 0  1  0  0  0  1  0 14]]
accuracy =  0.6910569105691057


In [63]:
model_path = "../models/"
model_name = "typeClassify_M2.pickle"
modelM = DataManager(model_path)
print(model_name)

typeClassify_M2.pickle


In [64]:
modelM.save_data(model_name, lr)

success save : ../models/typeClassify_M2.pickle
